商品名が似ているものはきっと同じdepartment_idなのでそれを特徴に加えたい  
愚直にコサイン類似度を計算すると時間がかかるので弊社オープンソースのNGTを使う  
＊初手ループさせまくったらtrainだけで30時間かかる見込みだった  

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/kaggle/input/yjfasttext/fasttext_gensim_model.pkl
/kaggle/input/ykc-2nd/sample_submission.csv
/kaggle/input/ykc-2nd/test.csv
/kaggle/input/ykc-2nd/train.csv
/kaggle/input/ykc-2nd/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
train = pd.read_csv('/kaggle/input/ykc-2nd/train.csv')
test = pd.read_csv('/kaggle/input/ykc-2nd/test.csv')

print(f'train: {train.shape}')
print(f'test: {test.shape}')

train: (24842, 6)
test: (24843, 5)


In [3]:
train.head()

,product_id,product_name,order_rate,order_dow_mode,order_hour_of_day_mode,department_id
0,0,All-Seasons Salt,2.779494e-06,0,11,12
1,1,Smart Ones Classic Favorites Mini Rigatoni Wit...,1.037875e-05,0,16,0
2,2,Green Chile Anytime Sauce,4.731054e-07,0,11,12
3,3,Dry Nose Oil,2.365527e-07,6,10,10
4,4,Pure Coconut Water With Orange,9.166416e-07,3,14,6


In [4]:
model = pd.read_pickle("/kaggle/input/yjfasttext/fasttext_gensim_model.pkl") 


def to_vec(x, model):
    unused_words = defaultdict(int)
    v = np.zeros(model.vector_size)
    for idx, w in enumerate(x):
        try:
            v += model[w] ## 単語が訓練済みモデルのvocabにあったら
        except:
            unused_words[w] += 1 ## ベクトルが存在しなかった単語をメモ
    v = v / (np.sqrt(np.sum(v ** 2)) + 1e-16) ## 長さを1に正規化
    return v 

def fe_vec(data, target_col):
    vecs = data[target_col].apply(lambda x : to_vec(x, model))
    vecs = np.vstack(vecs)

    fasttext_pretrain_cols = [f"fasttext_{target_col}_{k}" for k in range(vecs.shape[1])]
    vec_df = pd.DataFrame(vecs, columns=fasttext_pretrain_cols)
    data = pd.concat([data, vec_df], axis = 1)
    return data
    
    
def fe_pipeline(train, test):
    train['is_test'] = 0
    test['is_test'] = 1

    data = pd.concat([train, test], sort=False)
    data.reset_index(drop=True, inplace=True)

    data["product_name_origin"] = data["product_name"].copy()
    data["product_name"] = data["product_name"].apply(lambda words : words.lower().replace(",", "").replace("&", "").split(" "))

    data = fe_vec(data, 'product_name')

    train = data[data['is_test']==0].reset_index(drop=True)
    test = data[data['is_test']==1].reset_index(drop=True)
    del train['is_test']
    del test['is_test']
    
    print('fe_pipeline done, train: {}, test: {}'.format(train.shape, test.shape))
    return train, test

In [5]:
train, test = fe_pipeline(train, test)

train.head()

fe_pipeline done, train: (24842, 307), test: (24843, 307)


,product_id,product_name,order_rate,order_dow_mode,order_hour_of_day_mode,department_id,product_name_origin,fasttext_product_name_0,fasttext_product_name_1,fasttext_product_name_2,...,fasttext_product_name_290,fasttext_product_name_291,fasttext_product_name_292,fasttext_product_name_293,fasttext_product_name_294,fasttext_product_name_295,fasttext_product_name_296,fasttext_product_name_297,fasttext_product_name_298,fasttext_product_name_299
0,0,"[all-seasons, salt]",2.779494e-06,0,11,12.0,All-Seasons Salt,-0.016038,-0.005413,-0.030873,...,-0.057035,0.031508,0.005112,0.000568,-0.008253,0.040396,0.056868,0.071870,-0.009923,-0.000702
1,1,"[smart, ones, classic, favorites, mini, rigato...",1.037875e-05,0,16,0.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,-0.044100,-0.020597,-0.012222,...,0.002409,-0.017017,-0.006144,0.038623,0.002476,-0.041209,0.012066,0.092758,0.062074,-0.079980
2,2,"[green, chile, anytime, sauce]",4.731054e-07,0,11,12.0,Green Chile Anytime Sauce,-0.018035,0.048657,0.011118,...,0.077602,-0.022785,-0.004824,0.016669,-0.038935,-0.043521,0.035625,0.018629,0.042928,0.042913
3,3,"[dry, nose, oil]",2.365527e-07,6,10,10.0,Dry Nose Oil,-0.051420,-0.023629,-0.048042,...,-0.048886,-0.037484,0.001347,-0.010236,0.002433,0.105475,-0.012166,0.042813,0.053351,0.032356
4,4,"[pure, coconut, water, with, orange]",9.166416e-07,3,14,6.0,Pure Coconut Water With Orange,0.027223,-0.002388,-0.010507,...,-0.027605,0.020973,0.026813,-0.031821,-0.015133,0.045930,0.020277,0.069537,0.072880,0.021491


## NGT

In [6]:
# https://github.com/yahoojapan/NGT/blob/master/python/README-jp.md
!pip3 install ngt

     |████████████████████████████████| 2.2 MB 2.8 MB/s 


In [7]:
fasttext_pretrain_cols = [f"fasttext_product_name_{k}" for k in range(300)]
lookup = train[fasttext_pretrain_cols].values
test_lookup = test[fasttext_pretrain_cols].values
lookup_department_id = train['department_id'].values
print(lookup.shape)
print(test_lookup.shape)
print(lookup_department_id.shape)

print(lookup[0, :10])

(24842, 300)
(24843, 300)
(24842,)
[-0.01603792 -0.0054128  -0.03087299  0.04006138  0.05883911 -0.02522631
  0.06385096 -0.0720704  -0.03872489 -0.00317417]


In [8]:
import ngtpy
import random

In [9]:
%%time

# trainのindex作成
index_path = 'ykc-2nd.anng'
    
dim = 300
ngtpy.create(index_path, dim, distance_type='Cosine') 
index = ngtpy.Index(index_path)

for idx in tqdm(range(len(lookup))):
    object = lookup[idx].tolist()
    index.insert(object)
        
index.build_index()
index.save()

# testのindex作成
test_index_path ='ykc-2nd-test.anng'

ngtpy.create(test_index_path, dim, distance_type='Cosine') 
index = ngtpy.Index(test_index_path)

for idx in tqdm(range(len(test_lookup))):
    object = test_lookup[idx].tolist()
    index.insert(object)
        
index.build_index()
index.save()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



CPU times: user 29.2 s, sys: 313 ms, total: 29.5 s
Wall time: 13.2 s


In [10]:
train_index = ngtpy.Index('ykc-2nd.anng')
index = ngtpy.Index(index_path)

query_id = 0
query_object = index.get_object(query_id)

# train_indexから似ている商品名の上位5件
result = train_index.search(query_object, size=6)
for rank, object in enumerate(result):
    print('{}\t{}\t{:.6f}\t{}'.format(rank + 1, object[0], object[1], train.product_name.loc[object[0]]))

1	0	0.000000	['all-seasons', 'salt']
2	10048	0.102711	['salt']
3	4526	0.139954	['premium', 'epsom', 'salt']
4	18098	0.169493	['iodized', 'sea', 'salt']
5	22159	0.172833	['iodized', 'salt', '', 'pepper']
6	19673	0.173879	['sea', 'salt']


In [11]:
def fe_run_ngt(df, index_path):
    train_index = ngtpy.Index('ykc-2nd.anng')
    index = ngtpy.Index(index_path)
    
    similar_dep_ids = np.zeros((len(df), 50))

    for query_id in tqdm(range(len(df))):
        query_object = index.get_object(query_id)
        
        # train_indexから似ている商品名の上位N件
        result = train_index.search(query_object, size=51)

        for rank, object in enumerate(result):
            if rank == 0:
                continue
                
            # 似ている商品名のdepartment_idを入れる
            similar_dep_ids[query_id, rank-1] = lookup_department_id[object[0]]
            
    return similar_dep_ids

In [12]:
%%time

train_similar_dep_ids = fe_run_ngt(train, 'ykc-2nd.anng')
test_similar_dep_ids = fe_run_ngt(test, 'ykc-2nd-test.anng')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



CPU times: user 1min 7s, sys: 945 ms, total: 1min 8s
Wall time: 1min 8s


In [13]:
cols = [f'similar_{n+1}' for n in range(50)]
train_similar_dep_df = pd.DataFrame(columns=cols, data=train_similar_dep_ids)
test_similar_dep_df = pd.DataFrame(columns=cols, data=test_similar_dep_ids)
train_similar_dep_df.to_csv('trin_cimilar.csv')
test_similar_dep_df.to_csv('test_similar_dep_df.csv')

print(train_similar_dep_df.shape)
print(test_similar_dep_df.shape)

(24842, 50)
(24843, 50)


In [14]:
train_similar_dep_df.head()

,similar_1,similar_2,similar_3,similar_4,similar_5,similar_6,similar_7,similar_8,similar_9,similar_10,...,similar_41,similar_42,similar_43,similar_44,similar_45,similar_46,similar_47,similar_48,similar_49,similar_50
0,12.0,10.0,12.0,12.0,12.0,10.0,12.0,18.0,12.0,12.0,...,12.0,12.0,12.0,12.0,18.0,12.0,0.0,18.0,15.0,12.0
1,0.0,0.0,0.0,0.0,8.0,8.0,20.0,8.0,0.0,0.0,...,14.0,11.0,8.0,8.0,0.0,8.0,0.0,8.0,8.0,7.0
2,5.0,5.0,5.0,5.0,5.0,5.0,12.0,5.0,12.0,5.0,...,5.0,5.0,14.0,5.0,5.0,0.0,12.0,8.0,12.0,12.0
3,10.0,10.0,10.0,10.0,10.0,12.0,12.0,10.0,10.0,10.0,...,10.0,10.0,10.0,12.0,12.0,10.0,12.0,16.0,10.0,12.0
4,6.0,6.0,6.0,6.0,20.0,6.0,6.0,6.0,6.0,6.0,...,6.0,10.0,10.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0


In [15]:
test_similar_dep_df.head()

,similar_1,similar_2,similar_3,similar_4,similar_5,similar_6,similar_7,similar_8,similar_9,similar_10,...,similar_41,similar_42,similar_43,similar_44,similar_45,similar_46,similar_47,similar_48,similar_49,similar_50
0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,2.0,...,18.0,18.0,18.0,13.0,18.0,18.0,18.0,0.0,18.0,18.0
1,6.0,6.0,6.0,5.0,6.0,6.0,6.0,20.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
2,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,6.0,0.0,20.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
3,0.0,20.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,0.0,6.0,6.0,6.0,3.0,3.0,6.0,20.0,6.0,6.0
4,2.0,15.0,12.0,15.0,2.0,2.0,2.0,2.0,20.0,12.0,...,0.0,12.0,0.0,18.0,0.0,18.0,20.0,0.0,2.0,12.0


In [16]:
count_df = train_similar_dep_df.progress_apply(lambda x: x.value_counts(), axis=1).fillna(0) 
top_n_df = count_df.progress_apply(lambda s: s.nlargest(5).index.tolist(), axis=1)
top_n_df = pd.DataFrame(top_n_df, columns=['top_n_list'])
train_top_n_df = pd.DataFrame(top_n_df['top_n_list'].to_list(), columns=['top1', 'top2', 'top3', 'top4', 'top5'])

In [17]:
count_df = test_similar_dep_df.progress_apply(lambda x: x.value_counts(), axis=1).fillna(0) 
top_n_df = count_df.progress_apply(lambda s: s.nlargest(5).index.tolist(), axis=1)
top_n_df = pd.DataFrame(top_n_df, columns=['top_n_list'])
test_top_n_df = pd.DataFrame(top_n_df['top_n_list'].to_list(), columns=['top1', 'top2', 'top3', 'top4', 'top5'])

In [18]:
train_top_n_df.head()

,top1,top2,top3,top4,top5
0,12.0,18.0,10.0,0.0,2.0
1,0.0,8.0,7.0,14.0,20.0
2,5.0,12.0,8.0,14.0,0.0
3,10.0,12.0,16.0,14.0,0.0
4,6.0,10.0,20.0,12.0,17.0


In [19]:
test_top_n_df.head()

,top1,top2,top3,top4,top5
0,18.0,0.0,2.0,12.0,13.0
1,6.0,20.0,5.0,0.0,1.0
2,6.0,20.0,5.0,14.0,0.0
3,6.0,20.0,0.0,3.0,17.0
4,12.0,2.0,0.0,18.0,15.0


In [20]:
train_top_n_df.to_csv('train_top_n.csv')
test_top_n_df.to_csv('test_top_n.csv')